In [2]:
pip install requests bs4

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl (205 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached beaut

# Homework Starter — Stage 04: Data Acquisition and Ingestion


Name: Swastik Mishra

Date: 08/12/2025

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [6]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents = True, exist_ok = True)
load_dotenv("../../project/.env"); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [7]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull

In [11]:
js

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2025-08-15',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2025-08-15': {'1. open': '234.0000',
   '2. high': '234.2800',
   '3. low': '229.3350',
   '4. close': '231.5900',
   '5. volume': '56038657'},
  '2025-08-14': {'1. open': '234.0550',
   '2. high': '235.1200',
   '3. low': '230.8500',
   '4. close': '232.7800',
   '5. volume': '51916275'},
  '2025-08-13': {'1. open': '231.0700',
   '2. high': '235.0000',
   '3. low': '230.4300',
   '4. close': '233.3300',
   '5. volume': '69878546'},
  '2025-08-12': {'1. open': '228.0050',
   '2. high': '230.8000',
   '3. low': '227.0700',
   '4. close': '229.6500',
   '5. volume': '55672301'},
  '2025-08-11': {'1. open': '227.9200',
   '2. high': '229.5600',
   '3. low': '224.7600',
   '4. close': '227.1800',
   '5. volume': '61806132'},
  '2025-08-08': {'1. open': '220

In [12]:
SYMBOL = 'AAPL'
url = 'https://www.alphavantage.co/query'
params = {'function':'TIME_SERIES_DAILY','symbol' : SYMBOL,'outputsize' : 'compact', 'apikey': os.getenv('ALPHAVANTAGE_API_KEY')}
r = requests.get(url, params = params, timeout = 30)
r.raise_for_status()
js = r.json()
key = [k for k in js if 'Time Series' in k][0]
df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns = {'index' : 'date','4. close' : 'close'})[['date' , 'close']]
df_api['date'] = pd.to_datetime(df_api['date'])
df_api['close'] = pd.to_numeric(df_api['close'])
v_api = validate(df_api, ['date','adj_close'])
v_api

{'missing': ['adj_close'], 'shape': (100, 2), 'na_total': 0}

In [13]:
_ = save_csv(df_api.sort_values('date'), prefix = 'api', source = 'alpha', symbol = SYMBOL)

Saved data/raw/api_source-alpha_symbol-AAPL_20250817-223747.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [20]:
rows

[['Company', 'Contact', 'Country'],
 ['Alfreds Futterkiste', 'Maria Anders', 'Germany'],
 ['Centro comercial Moctezuma', 'Francisco Chang', 'Mexico'],
 ['Ernst Handel', 'Roland Mendel', 'Austria'],
 ['Island Trading', 'Helen Bennett', 'UK'],
 ['Laughing Bacchus Winecellars', 'Yoshi Tannamuri', 'Canada'],
 ['Magazzini Alimentari Riuniti', 'Giovanni Rovelli', 'Italy'],
 ['Tag', 'Description'],
 ['<table>', 'Defines a table'],
 ['<th>', 'Defines a header cell in a table'],
 ['<tr>', 'Defines a row in a table'],
 ['<td>', 'Defines a cell in a table'],
 ['<caption>', 'Defines a table caption'],
 ['<colgroup>',
  'Specifies a group of one or more columns in a table for formatting'],
 ['<col>',
  'Specifies column properties for each column within a <colgroup> element'],
 ['<thead>', 'Groups the header content in a table'],
 ['<tbody>', 'Groups the body content in a table'],
 ['<tfoot>', 'Groups the footer content in a table']]

In [24]:
SCRAPE_URL = 'https://www.w3schools.com/html/html_tables.asp'  # TODO: replace with permitted page
headers = {'User-Agent' : 'AFE-Homework/1.0'}

resp = requests.get(SCRAPE_URL, headers = headers, timeout = 30); resp.raise_for_status()
soup = BeautifulSoup(resp.text, 'html.parser')
rows = [[c.get_text(strip = True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
header, *data = [r for r in rows if r]
df_scrape = pd.DataFrame(data, columns = header)

# if 'Price' in df_scrape.columns:
#     df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns))
v_scrape

{'missing': [], 'shape': (17, 3), 'na_total': 11}

In [22]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data/raw/scrape_site-example_table-markets_20250817-224028.csv


## Documentation
- API Source: [(URL/endpoint/params)](https://www.alphavantage.co/query)
- Scrape Source: [(URL/table description)](https://www.w3schools.com/html/html_tables.asp)
- Assumptions & risks: W3 Schools Page format might change, AlphaVangtage might move API to paid tier.
- Confirm `.env` is not committed.